In [1]:
import pandas as pd
from replace_with_warning import replace_with_warning
from pandas_helper import broadcast_simple
import numpy as np

from res_ind_lib import average_over_rp

In [2]:
iso3_to_wb = pd.read_csv("inputs/iso3_to_wb_name.csv", index_col="iso3", squeeze=True)


#Names to WB names
any_to_wb = pd.read_csv("inputs/any_name_to_wb_name.csv",index_col="any",squeeze=True)


# AAL

## agg data

In [3]:
gar_aal_data = pd.read_csv("inputs/GAR15 results feb 2016_AALmundo.csv", encoding="latin-1", thousands=",", )

# .drop(["ISO","Country"],axis=1)

#these are part of france
gar_aal_data.ISO.replace(["GUF", "GLP", "MTQ", "MYT", "REU"],"FRA", inplace=True)

#these ara part of the UK
gar_aal_data.ISO.replace(["FLK", "GIB", "MSR"],"GBR", inplace=True)

#wb spellings
gar_aal_data = gar_aal_data.set_index(replace_with_warning(gar_aal_data.ISO,iso3_to_wb)).drop(["ISO","Country"],axis=1)
gar_aal_data.index.name="country"


#aggregates UK and France pieces to one country
gar_aal_data =  gar_aal_data.sum(level="country") 


#takes exposed value out
gar_exposed_value = gar_aal_data.pop("EXPOSED VALUE")


#rename hazards
gar_aal_data = gar_aal_data.rename(columns=lambda s:(s.lower()))
gar_aal_data= gar_aal_data.rename(columns = {"tropical cyclones":"cyclones", "riverine floods":"flood", "storm surge":"surge"})


gar_aal_data

AAL = (gar_aal_data.T/gar_exposed_value).T
AAL.head()

gar_aal_data.sum().sum()

C:\Users\WB451855\OneDrive - WBG\resilience_indicator_SIDS\replace_with_warning.py:21: UserWarning: These entries were not found in the dictionary:
ESH
  warnings.warn("These entries were not found in the dictionary:\n"+joiner.join(series_in[are_missing].unique()))


293387.32000000007

## wind and surge

In [4]:

def get_dk_over_k_from_file(path="inputs/GAR_data_surge.csv"):
    gardata_surge = pd.read_csv(path).dropna(axis=1, how="all")

    #these are part of france
    gardata_surge.Country.replace(["GUF", "GLP", "MTQ", "MYT", "REU"],"FRA", inplace=True)

    #these ara part of the UK
    gardata_surge.Country.replace(["FLK", "GIB", "MSR"],"GBR", inplace=True)


    gardata_surge = gardata_surge.set_index(replace_with_warning(gardata_surge.Country, iso3_to_wb))[["AAL", "Exposed Value (from GED)"]] # drop(["Country","Country name", "Region", "GA", "Hazard"],axis=1)

    # gardata_surge.rename(columns=lambda s: float(s[4:-5]))

    return gardata_surge["AAL"].sum(level=0)/gardata_surge["Exposed Value (from GED)"].sum(level=0)


aal_surge = get_dk_over_k_from_file("inputs/GAR_data_surge.csv")
aal_wind = get_dk_over_k_from_file("inputs/GAR_data_wind.csv")
aal_recomputed =aal_surge +aal_wind


((AAL.cyclones - aal_recomputed)/AAL.cyclones).abs().sort_values(ascending=False).head()




C:\Users\WB451855\OneDrive - WBG\resilience_indicator_SIDS\replace_with_warning.py:21: UserWarning: These entries were not found in the dictionary:
ESH
  warnings.warn("These entries were not found in the dictionary:\n"+joiner.join(series_in[are_missing].unique()))


Costa Rica                        1.000000e+00
Tonga                             3.812629e-01
St. Vincent and the Grenadines    2.115747e-16
Venezuela, RB                     2.115004e-16
United States                     2.085513e-16
dtype: float64

In [5]:
c="Costa Rica"
f = (aal_surge/aal_recomputed).mean()
aal_surge.ix[c] = f*AAL.cyclones[c]
aal_wind.ix[c] = (1-f)*AAL.cyclones[c]


## AAL splitted

In [6]:
AAL_splitted = AAL.drop("cyclones",axis=1).assign(wind= aal_wind, surge = aal_surge).rename(columns=dict(floods="flood")).stack()
AAL_splitted.index.names=["country", "hazard"]

# PMLs and Exposed value

In [7]:
gardata =pd.read_csv("inputs/GAR15 results feb 2016_PML mundo.csv", encoding="latin-1", header=[0,1,2], index_col=0)#.sort_index(axis=1)
gardata.index.name = "country"

gardata = gardata.reset_index()

gardata.columns.names=["hazard","rp","what"]

#these are part of france
gardata.country = gardata.country.replace(["French Guiana", "Guadeloupe", "Martinique", "Mayotte", "Réunion"],"France")

#these ara part of the UK
gardata.country = gardata.country.replace(["Falkland Islands (Malvinas)", "Gibraltar", "Montserrat"],"United Kingdom")

gardata.country = replace_with_warning(gardata.country, any_to_wb.dropna())

gardata =gardata.set_index("country")

#aggregates UK and France
# gardata = gardata.sum(level="country", axis=1)


#Other format for zero
gardata = gardata.replace("---",0).astype(float)

#aggregates france and uk
gardata = gardata.sum(level="country")

gardata.head()


C:\Users\WB451855\OneDrive - WBG\resilience_indicator_SIDS\replace_with_warning.py:21: UserWarning: These entries were not found in the dictionary:
Western Sahara
  warnings.warn("These entries were not found in the dictionary:\n"+joiner.join(series_in[are_missing].unique()))


hazard         EXPOSED VALUE  EARTHQUAKE                                      \
rp                  National          20                50               100   
what             million US$ million US$   [%] million US$   [%] million US$   
country                                                                        
Afghanistan         60187.90      317.83  0.53      637.57  1.06     1034.61   
Albania             40459.70      112.83  0.28      239.07  0.59      412.61   
Algeria            899206.00     2965.76  0.33     5719.85  0.64     8834.51   
American Samoa       1930.49        0.39  0.02        2.46  0.13        7.18   
Andorra              8381.65        0.19  0.00        0.39  0.00        0.64   

hazard                                                ...    Riverine floods  \
rp                           250               500    ...                100   
what             [%] million US$   [%] million US$    ...        million US$   
country                                               ...                      
Afghanistan     1.72     1800.74  2.99     2585.25    ...        1304.836191   
Albania         1.02      817.76  2.02     1308.38    ...         559.456815   
Algeria         0.98    14487.39  1.61    19624.90    ...        2455.765743   
American Samoa  0.37       21.09  1.09       39.03    ...                NaN   
Andorra         0.01        1.59  0.02        3.66    ...                NaN   

hazard                                                                  \
rp                                250                    500             
what                 [%]  million US$       [%]  million US$       [%]   
country                                                                  
Afghanistan     2.167938  1662.178063  2.761648  1749.645544  2.906972   
Albania         1.382751   985.990186  2.436969  1152.001339  2.847281   
Algeria         0.273104  5227.693307  0.581368  6507.867777  0.723735   
American Samoa       NaN          NaN       NaN          NaN       NaN   
Andorra              NaN          NaN       NaN          NaN       NaN   

hazard                                                        
rp                     1000                   1500            
what            million US$       [%]  million US$       [%]  
country                                                       
Afghanistan     1924.580505  3.197620  2099.515466  3.488268  
Albania         1342.639991  3.318463  1434.570014  3.545676  
Algeria         7652.708961  0.851052  8032.451716  0.893283  
American Samoa          NaN       NaN          NaN       NaN  
Andorra                 NaN       NaN          NaN       NaN  

[5 rows x 71 columns]

In [8]:

#Looks at the exposed value, for comparison
exposed_value_GAR = gardata.pop("EXPOSED VALUE").squeeze()
# df["exposed_value_GAR"].head()


#rename hazards
gardata = gardata.rename(columns=lambda s:(s.lower()))
gardata= gardata.rename(columns = {"wiind":"wind", "riverine floods":"flood", "storm surge":"surge"})

#rp to floats
def str_to_float(s):
    try:
        return (float(s))
    except ValueError:
        return s
gardata = gardata.rename(columns=str_to_float)

gardata.head()



#Asset losses per event
dK_gar = gardata.swaplevel("what","hazard",axis=1)["million us$"].swaplevel("rp","hazard",axis=1).stack(level=["hazard","rp"], dropna=False).sort_index()

#Exposed value per event
ev_gar=  broadcast_simple(exposed_value_GAR,dK_gar.index)

#Fraction of value destroyed
frac_value_destroyed_gar = dK_gar/ev_gar

frac_value_destroyed_gar.dropna().unstack("country").shape

frac_value_destroyed_gar.to_csv("intermediate/frac_value_destroyed_gar.csv", encoding="utf-8", header=True)

frac_value_destroyed_gar.head()

country      hazard      rp   
Afghanistan  earthquake  20.0     0.005281
                         50.0     0.010593
                         100.0    0.017190
                         250.0    0.029919
                         500.0    0.042953
dtype: float64

In [9]:
#Check. should be 0
(exposed_value_GAR - gar_exposed_value).abs().sort_values(ascending=True).head(5)

country
Afghanistan      0.0
New Caledonia    0.0
New Zealand      0.0
Nicaragua        0.0
Niger            0.0
dtype: float64

# Combines exposed value to frac destroyed from events

In [10]:
capital_losses_from_GAR_events = pd.read_csv("intermediate/capital_losses_from_GAR_events.csv", index_col=["country","hazard","rp"], squeeze=True)

FileNotFoundError: File b'intermediate/capital_losses_from_GAR_events.csv' does not exist

In [ ]:
frac_cap_distroyed_from_events = capital_losses_from_GAR_events/ev_gar.median(level="country")
frac_cap_distroyed_from_events.to_csv("intermediate/frac_cap_distroyed_from_events.csv", header=True)



In [ ]:
(average_over_rp(frac_cap_distroyed_from_events).squeeze()/ AAL_splitted ).replace(np.inf,np.nan).dropna().sort_values()


# adds event to PML to complete AAL  (old bad method)

In [ ]:
frac_value_destroyed_gar.head()

In [ ]:
AAL_splitted.head()

 how much greater AAL is than sum of PML

In [ ]:
paf= (AAL_splitted / average_over_rp(frac_value_destroyed_gar).squeeze()).replace(np.inf,np.nan).sort_values(ascending=False)
paf[paf>5]

In [ ]:
paf.dropna().tail(30)

In [ ]:
frac_value_destroyed_gar["Japan"]["flood"]*1e6

## add frequent events

In [ ]:
last_rp = 20
new_rp = 1

added_proba = 1/new_rp - 1/last_rp

new_frac_destroyed = (AAL_splitted - average_over_rp(frac_value_destroyed_gar).squeeze())/(added_proba)
   
#REMOVES 'tsunamis' and 'earthquakes' from this thing
# new_frac_destroyed = pd.DataFrame(new_frac_destroyed).query("hazard not in ['tsunami', 'earthquake']").squeeze()
new_frac_destroyed.describe(), new_frac_destroyed.sample(10)


In [ ]:
hop = frac_value_destroyed_gar.unstack()
hop[new_rp]=   new_frac_destroyed
hop= hop.sort_index(axis=1)
hop.head()


frac_value_destroyed_gar_completed = hop.stack()
frac_value_destroyed_gar_completed.head()  #this shows earthaquke was not updated

In [ ]:
frac_value_destroyed_gar_completed.sample(10)

### double check. expecint zeroes expect for quakes and tsunamis

In [ ]:
(average_over_rp(frac_value_destroyed_gar_completed).squeeze() - AAL_splitted).abs().sort_values(ascending=False).sample(10)

### places where new values are higher than values for 20

In [ ]:
test = frac_value_destroyed_gar_completed.unstack().replace(0,np.nan).dropna().assign(test=lambda x:x[new_rp]/x[20]).test

max_relative_exp = .8

overflow_frequent_countries = test[test>max_relative_exp].index
print("overflow in {n} (country, event)".format(n=len(overflow_frequent_countries)))

test[overflow_frequent_countries].sort_values(ascending=False)

## for this places, add infrequent events


In [ ]:
hop=frac_value_destroyed_gar_completed.unstack()
hop[1]=hop[1].clip(upper=max_relative_exp*hop[20])
frac_value_destroyed_gar_completed = hop.stack()

In [ ]:
new_rp = 2000

added_proba = 1/2000

new_frac_destroyed = (AAL_splitted - average_over_rp(frac_value_destroyed_gar_completed).squeeze())/(added_proba)
   
#REMOVES 'tsunamis' and 'earthquakes' from this thing
# new_frac_destroyed = pd.DataFrame(new_frac_destroyed).query("hazard in ['tsunami', 'earthquake']").squeeze()
new_frac_destroyed.describe(), new_frac_destroyed.sample(10)

In [ ]:
hop = frac_value_destroyed_gar.unstack()
hop[new_rp]=   new_frac_destroyed
hop= hop.sort_index(axis=1)
hop.head()


frac_value_destroyed_gar_completed = hop.stack()
frac_value_destroyed_gar_completed.head(10)  

In [ ]:
test = frac_value_destroyed_gar_completed.unstack().replace(0,np.nan).dropna().assign(test=lambda x:x[new_rp]/x[1500]).test
test[test>1.1].sort_values(ascending=False)

In [ ]:
test[test>2].shape, test.shape

In [ ]:
frac_value_destroyed_gar_completed["United States"]["earthquake"]

In [ ]:
AAL_splitted.ix["United States"]*100

In [ ]:
pd.DataFrame((average_over_rp(frac_value_destroyed_gar).squeeze()/AAL_splitted).replace(0,np.nan).dropna().sort_values())